In [17]:
import pandas as pd
import geopandas as gpd

In [2]:
from src.leaflet_creation import create_map_visualization, get_color_for_df, gradient_color

In [3]:
seg = gpd.read_file('data/segments_dc.geojson')

In [4]:
def getSafetyColor(d):
    if pd.isnull(d): return '#ffffffff'
    if d > 4: return '#1a9641'  # green (4, ]
    if d > 3: return '#a6d96a'  # light green (3, 4]
    if d > 2: return '#ffffbf'  # yellow: (2, 3]
    if d > 1: return '#fdae61'  # orange: (1, 2]
    return '#d7191c'            # red: (, 1]


In [5]:
%matplotlib inline

# pred for ground truth

In [4]:
true_pred = pd.read_csv('poster used true and pred.csv',index_col=0)

In [7]:
simplified = seg[['geometry']].merge(true_pred, left_index=True, right_index=True, how='right')
simplified['diff'] = simplified.apply(lambda x: abs(x.csl-x.pred), axis=1)
true = simplified[['geometry', 'csl']].reset_index()
pred = simplified[['geometry', 'pred']].reset_index()
diff = simplified[['geometry', 'diff']].reset_index()
diff['diff'].value_counts()

In [24]:
true['color'] = true.csl.apply(getColor)
pred['color'] = pred.pred.apply(getColor)
diff['color'] = diff['diff'].apply(lambda x: getColor(5-x))

In [25]:
html_title = 'poster map'
file_path = ''
file_name = 'poster map'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets', 'stsg']
map_layers = ['light','streets', 'satellite']
binding_data=[['true_map','actual map'],['pred_map','predicted map'],['diff_map', 'true-pred diff']]
gpdfs = [true, pred, diff]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs, width=1000)

# pred whole dc only

In [18]:
scheme_name='default'
scheme_name='experience'

pred_whole_dc = pd.read_csv('data/Pred_y_whole_dc_5lvl_spatial_%s.csv' % scheme_name,index_col=0)

whole = seg[['geometry']].merge(pred_whole_dc, left_index=True, right_index=True, how='right').reset_index()

whole['color'] = whole['whole-pred'].apply(getColor)

In [19]:
html_title = 'DC pred map ' + scheme_name
file_path = ''
file_name = 'DC pred map ' + scheme_name
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['osm_mapnik', 'stsg']
map_layers = ['dark_black','osm_mapnik']
binding_data=[['pred_map','predicted map']]
gpdfs = [whole]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs, width=2000, height=1500)

# pred whole dc with prob

In [23]:
import json 
re_geo = json.load(open('data/reversed_geocoding_midpoint_dc.json'))

In [47]:
display = [x['display_name'].replace(' Washington, District of Columbia,', '').replace(', United States of America','') for x in re_geo]

In [48]:
display

['5030, 1st Street Northwest, Fort Totten,  20011',
 '5216, 1st Street Northwest, Brightwood Park, Fort Totten,  20011',
 '5317, 1st Street Northwest, Brightwood Park, Fort Totten,  20011',
 '5415, 1st Street Northwest, Brightwood Park, Manor Park,  20011',
 '5514, 1st Street Northwest, Brightwood Park, Manor Park,  20011',
 '5614, 1st Street Northwest, Brightwood Park, Manor Park,  20011',
 '5710, 1st Street Northwest, Brightwood Park, Manor Park,  20011',
 '5723, 1st Street Northwest, Brightwood Park, Manor Park,  20011',
 '6521, 1st Street Northwest, Lamond Riggs,  20012',
 '6624, 1st Street Northwest, Lamond Riggs,  20012',
 '1165, 1st Place Northwest, North Capitol Street, Penn Quarter,  20004',
 'Triumphant Christian Church, 5408, 1st Place Northwest, Manor Park,  20011',
 '5614, 1st Place Northwest, Manor Park,  20011',
 'North Capitol Street, K Street Northwest, North Capitol Street, Penn Quarter,  20001',
 '1173, 1st Terrace Northwest, North Capitol Street, Penn Quarter,  2000

In [6]:
scheme_name = 'experience'
pred_with_prob = pd.read_csv('data/Pred_y_with_prob_whole_dc_5lvl_spatial_experience.csv' ,index_col=0)

In [7]:
pred_with_prob['color'] = pred_with_prob.pred_y.apply(getSafetyColor)

In [8]:
pred_with_prob['proba_color'] = pred_with_prob.probability.apply(gradient_color)

In [29]:
pred_with_prob['display'] = display

In [30]:
map_gpdf = seg[['geometry']].join(pred_with_prob.drop('true_y', axis=1), how='right').reset_index()

In [31]:
map_gpdf = map_gpdf[map_gpdf.index_seg!=11813]

In [35]:
true_map = seg[['geometry']].join(pred_with_prob[~pred_with_prob.true_y.isnull()][['true_y', 'display']], how='right')

In [36]:
true_map['color'] = true_map.true_y.apply(getSafetyColor)

In [38]:
true_map.columns=['geometry', 'Ground Truth Level', 'display', 'color']

In [39]:
one_v_per_seg = pd.read_csv('data/video per segment - Largest ratio.csv', index_col=0)

In [41]:
true_map = true_map.join(one_v_per_seg)

In [15]:
with open('pred.geojson', 'w') as f:
    f.write(map_gpdf.to_json())

In [16]:
with open('true.geojson', 'w') as f:
    f.write(true_map.to_json())

In [22]:
# true_map.join(one_v_per_seg).to_json()

In [42]:
html_title = 'new DC pred map with prob' + scheme_name
file_path = ''
file_name = 'new DC pred map with prob' + scheme_name
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['osm_mapnik', 'stsg']
map_layers = ['dark_black','osm_mapnik']
binding_data=[['pred_map','predicted map'], ['true_map', 'Ground Truth Map']]
gpdfs = [map_gpdf,true_map]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)